
Before using this tutorial, ensure that the following are on your system 

1. SteganoGAN is installed (pip install steganogan)

2. CUDA is availabe on machine if you are training on a large dataset

3. Your dataset is available on your machine and split into its Train and Validation folders. Practice datasets are available via the data/ folder script download.sh


In the next code step, we will import all the necessary components from SteganoGAN. Let us go through each of these components.


In [25]:
import numpy as np #numpy is used for a parameter input

This imports the SteganoGAN class which has wrapper functions that allow you to use SteganoGAN. This wrapper class includes methods to create a SteganoGAN architecture, train that architecture, save the trained model, and then load and use trained models.


In [26]:
from steganogan import SteganoGAN

The dataloader is used to load images from a selected database with different specified hyper parameters. You can tune these parameters when loading the images from the database. 


In [27]:
from steganogan.loader import DataLoader

The encoders are the architectural models that are used to encode the messages inside the image. A basic encoder uses less layers compared to the dense encoder and is thereby faster to train but less robust. The encoder represents a PyTorch architecture. 

Any steganoGAN model will have ONE encoder that will either be a BasicEncoder or DenseEncoder. You may choose which architecture to use.


In [28]:
from steganogan.encoders import BasicEncoder, DenseEncoder

The decoders are the architectural models that are used to decode the messages inside the image. A basic decoder uses less layers compared to the dense decoder and is thereby faster to train but less robust. The decoder represents a PyTorch architecture. 

Any steganoGAN model will have ONE decoder that will either be a BasicDecoder or DenseDecoder. You may choose which architecture to use.


In [29]:
from steganogan.decoders import BasicDecoder, DenseDecoder

The Critic measures how well the information is hidden inside the image. The Critic ensures that the images are well hid. The Critic acts as the discriminator in the GAN system. 

SteganoGAN only uses a BasicCritic. This parameter will never be changed 


In [30]:
from steganogan.critics import BasicCritic

In this next cell, we load in the data for our training and validation dataset. The training dataset is used to train the model while the validation dataset is used to ensure that the model is working correctly. There are several parameters that can you choose to tune.

1. The first parameter represents the path to the directory. This can be a relative path or an absolute path from the notebook file. 

2. The limit parameter represents the amount of images you wish to use. If limit is set as np.inf, all the images in the directory will be used.

3. The shuffle parameter allows you to randomly shuffle the images in the directory when using them. This is good for introducing better randomness into your training.

4. The batch_size allows you to indicste how many images should be processed in one batch. Larger batch processing usually leads to faster computation times at the loss of some accuracy. Thus, there is a tradeoff to be made here. You can tune this parameter. 



In [31]:
# Load the data
train = DataLoader('data/div2k/train/', limit=np.inf, shuffle=True, batch_size=4)


validation = DataLoader('data/div2k/val/', limit=np.inf, shuffle=True, batch_size=4)

Below we are deciding on the architecture that we want to use for SteganoGAN. There are several parameters that you can tune here to decide on the architecture. Let us go over them:

1. The first parameter represents the data_depth. Data depth represents how many layers we want to represent the data with. Currently, data is representated as a N x data_depth x H x W. Usually, we set this to 1 since that suffices for our needs. For more robustness set this data depth to a higher number.

2. We choose the encoder instance. You can choose either a BasicEncoder or DenseEncoder.

3. We choose the decoder instance. You can choose either a DenseEncoder or DenseDecoder.

4. We choose the critic instance. The only option is the BasicCritic

5. We choose the hidden_size. This represents the number of channels we wish to use in the hidden layers of our architecture. You can tune this parameter. We chose 32 as we find relatively good models with these number of channels.

6. We choose if we want to use cuda. This only works if the machine is CUDA-enabled

7. We choose if we want to use verbose. Verbose prints out all the information regarding training. 

In [32]:
# Create the SteganoGAN instance
steganogan = SteganoGAN(1, BasicEncoder, BasicDecoder, BasicCritic, hidden_size=32, cuda=True, verbose=True)

CUDA is not available. Defaulting to CPU device


Once the architecture has been decided and the training and validation data are we loaded, we can begin training. To train call this function with the number of epochs for you wish to train.

Note, we have set epochs=10 here so that you can train relatively fast with CUDA. On CUDA, this will take ~1 hour for a reasonable sized dataset. In practice, we use around 100 Epochs to properly train the model.

In [ ]:
# Fit on the given data
steganogan.fit(train, validation, epochs=10)

Once the model is trained, we save the model to a .steg file. In this file, we save all the model weights and the architectures that these weights compose. 

Both the encoder and decoder are saved in the same file. 

In [ ]:
# Save the fitted model
steganogan.save('models/basic.steg')

The next command loads a previously generated model. It takes a couple of different parameters. 

1. The architecture defines a pre-saved model that comes with the SteganoGAN package. You can choose basic or dense
2. The path can be used to point to your own specific model that you just created
3. Verbose parameter allows for information to be printed to the command line.

In [35]:
# Load the model
steganogan = SteganoGAN.load(architecture='basic', path=None, verbose=True)

CUDA is not available. Defaulting to CPU device


To encode an image, we pass in the input image, the output image name and the message we wish to encode. 

In [22]:
# Encode a message in input.png
steganogan.encode('input.png', 'output.png', 'This is a super secret message!')

Encoding completed.


To decode an image, we pass in the encoded image. Note that encoders and decoders come in pairs. A decoder can only decode an image that was encoded from its pair encoder. 

In [23]:
# Decode the message from output.png
steganogan.decode('output.png')

'This is a super secret message!'